## Training

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import itertools
from keras.callbacks import EarlyStopping
from keras import backend as K
from sklearn.metrics import f1_score
import tools_avg_01 as tools

training_data, training_label, validation_data, validation_label, validation_cate_label = tools.get_data()

ks = 5
num_layer = 10
bs = 30
lr = 0.0001

Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)
KFold(n_splits=5, random_state=100, shuffle=True)
trian: [    0     1     2 ... 10148 10149 10150] len 8120 test: [   12    21    22 ... 10136 10137 10138] len 2031
trian: [    2     4     5 ... 10148 10149 10150] len 8121 test: [    0     1     3 ... 10143 10144 10147] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [    4     5     8 ... 10130 10133 10146] len 2030
trian: [    0     1     2 ... 10146 10147 10149] len 8121 test: [   17    23    26 ... 10135 10148 10150] len 2030
trian: [    0     1     3 ... 10147 10148 10150] len 8121 test: [    2     6     7 ... 10142 10145 10149] len 2030


In [2]:
def run(bs, lr, ks, num_layer):
    
    for index, (X_train, Y_train, X_val, Y_val, val_cat) in enumerate(zip(training_data,
                                                       training_label,
                                                       validation_data,
                                                       validation_label,
                                                       validation_cate_label)):
        
        if index != 1: continue
        model = tools.create_model(lr, bs, ks, num_layer)
        print("Fold "+str(index))
        early_stop = EarlyStopping(patience=20)
        history = model.fit(x = X_train, 
                            y = Y_train,
                            epochs=100,
                            validation_data=(X_val, Y_val),
                            callbacks=[early_stop],
                            batch_size=bs, 
                            verbose=1)
        
        model.save_weights('my_model_weights.h5')
        model.save('K-1_model.h5')
        
#         evaluation = model.evaluate(x = X_val, y = Y_val)
#         validation_prediction = model.predict_classes(X_val, batch_size=bs)
#         score = f1_score(val_cat, validation_prediction, average=None)
#         print(score)
        return model
    
model = run(bs, lr, ks, num_layer)

Fold 1
Train on 8121 samples, validate on 2030 samples
Epoch 1/100
8121/8121 [==============================] - 15s 2ms/step - loss: 1.0463 - acc: 0.5658 - val_loss: 0.9608 - val_acc: 0.5946
Epoch 2/100
8121/8121 [==============================] - 8s 987us/step - loss: 0.9733 - acc: 0.5824 - val_loss: 0.9274 - val_acc: 0.5990
Epoch 3/100
8121/8121 [==============================] - 8s 996us/step - loss: 0.9096 - acc: 0.5917 - val_loss: 0.8780 - val_acc: 0.6084
Epoch 4/100
8121/8121 [==============================] - 8s 1ms/step - loss: 0.8488 - acc: 0.6286 - val_loss: 0.8007 - val_acc: 0.6961
Epoch 5/100
8121/8121 [==============================] - 9s 1ms/step - loss: 0.8027 - acc: 0.6582 - val_loss: 0.7311 - val_acc: 0.6768
Epoch 6/100
8121/8121 [==============================] - 8s 1ms/step - loss: 0.7354 - acc: 0.6940 - val_loss: 0.7122 - val_acc: 0.6773
Epoch 7/100
8121/8121 [==============================] - 8s 1ms/step - loss: 0.6991 - acc: 0.7107 - val_loss: 0.6155 - val_acc: 0.

In [5]:
import time
start_time = time.time()
validation_prediction = model.predict_classes(validation_data[0], batch_size=bs)
end_time = time.time() - start_time
print(end_time)
score = f1_score(validation_cate_label[0], validation_prediction, average=None)
print(score)

0.6705787181854248
[0.97721519 0.92424242 0.95069532 0.88812392]


## Inference

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# from keras.models import load_weights
from sklearn.metrics import f1_score
import tools

training_data, training_label, validation_data, validation_label, validation_cate_label = tools.get_data()
ks = 5
num_layer = 10
bs = 30
lr = 0.001
X_train, Y_train, X_val, Y_val, val_cat = training_data[1], training_label[1], validation_data[1], validation_label[1], validation_cate_label[1]

model = tools.create_model(lr, bs, ks, num_layer)
model.load_weights('my_model_weights.h5')
print("Model loaded!")

Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)
KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    2     4     5 ... 10148 10149 10150] len 8120 test: [    0     1     3 ... 10136 10138 10143] len 2031
trian: [    0     1     3 ... 10146 10147 10150] len 8121 test: [    2     4     9 ... 10145 10148 10149] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [   20    22    23 ... 10139 10141 10144] len 2030
trian: [    0     1     2 ... 10145 10148 10149] len 8121 test: [    5    17    18 ... 10146 10147 10150] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [   10    11    13 ... 10131 10137 10142] len 2030
Model loaded!


In [3]:
from keras.models import load_model
model = load_model('K-1_model.h5')

In [2]:
%time
evaluation = model.evaluate(x = X_val, y = Y_val)
validation_prediction = model.predict_classes(X_val, batch_size=bs)
score = f1_score(val_cat, validation_prediction, average=None)
print(score)

CPU times: user 7 µs, sys: 8 µs, total: 15 µs
Wall time: 36.5 µs
2030/2030 [==============================] - 2s 954us/step
[0.         0.06692532 0.03892944 0.00674536]


/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
